In [22]:
import psycopg2
import pandas as pd

In [102]:
class DataBase:
    def __init__(self, host, dbname, username, password, port):
        self.host = host 
        self.port = port 
        self.dbname = dbname 
        self.username = username 
        self.password = password 
    
    def __connect__(self):
        """Opens connector class and initiates cursor"""
        self.con = psycopg2.connect(host=self.host, port=self.port, user=self.username, password=self.password, 
                                                 database=self.dbname) 
        self.cur = self.con.cursor()

    def __disconnect__(self):
        """Commits any changes to the database and closes connection"""
        self.con.commit()
        self.con.close()
        
    def conn(self):
        self.con = psycopg2.connect(host=self.host, port=self.port, user=self.username, password=self.password, 
                                                 database=self.dbname) 
        return self.con

    def fetch(self, sql, variables=None):
        """Connects to database, fetches data specific to sql query, then disconnects from database"""
        self.__connect__()
        try:
            self.cur.execute(sql, variables)
            result = self.cur.fetchall()
            return result
        except Exception as e:
            print (e)
        finally:
            self.__disconnect__()
        

    def execute(self, sql, variables=None):
        """Connects to database, executes sql query, along with any variables, then disconnects from database"""
        self.__connect__()
        try:
            self.cur.execute(sql, variables)
        except Exception as e:
            print (e)
        finally:
            self.__disconnect__()
            
    def get_cols(self, table, details='no'):
        data = self.fetch("""
                SELECT *
          FROM information_schema.columns
        where table_schema = 'public'
             ;
                """)
        cols = []
        if details == 'yes':
            for i in data:
                if i[2:][0] == table:
                    print (i[2:][1], i[2:][5], i[2:][6])
        else:
            for i in data:
                if i[2:][0] == table:
                    cols.append(i[2:][1])
            str_cols = ', '.join(cols)
            return str_cols
    
    def get_tables(self):
        return self.fetch('''
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name;
            ''')

# Connect to database

In [103]:
username = 'postgres'
password = 'postgres'
host = 'boxchamp.csyqx1sutmfo.us-west-2.rds.amazonaws.com'
dbname = 'postgres'
port = 5432
db = DataBase(host, dbname, username, password, port)

# Create tables

In [105]:
# Create pets table
db.execute('''
    CREATE TABLE IF NOT EXISTS pets (
    petid varchar,
    name varchar,
    kind varchar,
    gender varchar,
    age int,
    ownerid varchar
)
''')

In [106]:
# Create owners owners
db.execute('''
CREATE TABLE IF NOT EXISTS owners (
    ownerid varchar,
    name varchar,
    surname varchar,
    streetaddress varchar,
    city varchar,
    state varchar(2),
    statefull varchar,
    zipcode varchar
)
''')

In [125]:
# Create owners proceduredetails
db.execute('''
CREATE TABLE proceduresdetails (
    proceduretype varchar,
    proceduresubcode varchar,
    description varchar,
    price float
)
''')

In [126]:
# Create owners prodecurehistory
db.execute('''
CREATE TABLE procedureshistory (
    petid varchar,
    proceduredate date,
    proceduretype varchar,
    proceduresubcode varchar
)
''')

In [127]:
# Check tables have been created
db.get_tables()

[('attendance',),
 ('class_time',),
 ('coach',),
 ('coach_schedule',),
 ('date',),
 ('lead',),
 ('member',),
 ('owners',),
 ('pets',),
 ('proceduresdetails',),
 ('procedureshistory',)]

# Copy data to tables

## With COPY statement if you have superuser rights on Postgres

In [ ]:
# Copy pets data
db.execute('''
    COPY pets FROM 'Pets.csv' DELIMITER ',' CSV HEADER
''')

In [ ]:
# Copy owners data
db.execute('''
    COPY owners FROM 'Owners.csv' DELIMITER ',' CSV HEADER
''')

In [ ]:
# Copy proceduredetails data
db.execute('''
    COPY proceduresdetails FROM 'ProceduresDetails.csv' DELIMITER ',' CSV HEADER
''')

In [ ]:
# Copy procedurehistory data
db.execute('''
    COPY procedureshistory FROM 'ProceduresHistory.csv' DELIMITER ',' CSV HEADER
''')

## With pandas.to_csv() 

An alternative if you do not have superuser access to your Postgres database

In [133]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}')

In [134]:
# Create pandas DataFrames
pets = pd.read_csv('Pets.csv')#, dtype={'OwnerID':'object'})
owners = pd.read_csv('Owners.csv')
proceduresdetails = pd.read_csv('ProceduresDetails.csv')
procedureshistory = pd.read_csv('ProceduresHistory.csv')

In [135]:
# Rename the columns to lower case to match database column names
pets = pets.rename(str.lower, axis='columns')
owners = owners.rename(str.lower, axis='columns')
proceduresdetails = proceduresdetails.rename(str.lower, axis='columns')
procedureshistory = procedureshistory.rename(str.lower, axis='columns')

In [137]:
# Copy data to database
pets.to_sql('pets', engine, if_exists='append', index=False)
owners.to_sql('owners', engine, if_exists='append', index=False)
proceduresdetails.to_sql('proceduresdetails', engine, if_exists='append', index=False)
procedureshistory.to_sql('procedureshistory', engine, if_exists='append', index=False)

In [138]:
# Verify # of records in each table
tables = ['pets','owners','proceduresdetails','procedureshistory']
for table in tables:
    db_count = db.fetch(f'''
        SELECT COUNT(*) FROM {table}
    ''')[0][0]
    print (db_count)

100
89
41
2284


In [139]:
dfs = [
    pets,
    owners,
    proceduresdetails,
    procedureshistory
    ]
for df in dfs:
    print (len(df))

100
89
41
2284
